In [1]:
import re
import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import linear_model
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

plt.rcParams['figure.figsize']=(12,6)

In [2]:
#realt = pd.read_csv('../data/realt.csv')
realt = pd.read_json('../parsing/realt_1612.json')

def get_realt_price(price):
    """
    Убирает дробную часть в цене
    """
    return int(''.join(re.findall(r'\d+', re.sub(r'[,.]\d+', ' ', price))))

# Удаление записи без цены
realt.drop(realt[realt['price_usa'] == 'Цена договорная'].index, inplace=True)

# Только Минск
realt = realt[realt['Населенный пункт'] == 'Минск']

# Исправление цены
realt['price_usa'] = realt['price_usa'].apply(get_realt_price)
realt['price_local'] = realt['price_local'].apply(get_realt_price)

realt.drop(realt[(realt['longitude'] > 27.8) | (realt['longitude'] < 27.3)].index, inplace=True)

drop_realt = ['УНП',
             'Район (области)',
             'Специалист',
             'Число спальных мест',
             'Направление',
             'На Новый год',
             'Вид этажа',
             'Год кап.ремонта', #много пропусков
             'Услуга агентства',
             'Срок сдачи', #?
             'Условия сдачи', #?
             'Населенный пункт',
             'Предоплата'
             ]

realt.drop(drop_realt, axis=1, inplace=True)

In [3]:
realt.isnull().sum(axis=0)

address                  0
area                     0
code                     0
description              0
floor                    0
kv                       0
latitude                 0
longitude                0
photos                 160
price_local              0
price_usa                0
views_day                0
views_week               0
zones                    0
Агентство             1259
Балкон                 806
Бытовая техника        777
Год постройки          449
Дата обновления          0
Дополнительно          829
Мебель                 186
Метро                  472
Планировка             361
Плита                  975
Полы                   414
Примечания            1062
Район города             0
Ремонт                 259
Сан/узел               725
Соседи по квартире    1953
Телефон                377
Тип дома               332
Этаж / этажность        12
dtype: int64

In [4]:
realt['description'].fillna('', inplace=True)
realt['Дополнительно'].fillna('', inplace=True)
realt['Примечания'].fillna('', inplace=True)
realt['photos'].fillna(0, inplace=True)
realt['Агентство'].fillna("нет", inplace=True)
realt['Балкон'].fillna("нет", inplace=True)
realt['Бытовая техника'].fillna('', inplace=True)
realt['Телефон'].fillna('нет', inplace=True)
realt['Мебель'].fillna('нет', inplace=True)
realt['Метро'].fillna('нет', inplace=True)
realt['Плита'].fillna('Пропуск', inplace=True)
realt['Ремонт'].fillna('Пропуск', inplace=True)
realt['Этаж / этажность'].fillna('0 / 0', inplace=True)

# кол-во комнат (0 - комната)
realt['rooms'] = realt['kv'].apply(lambda kv: 0 if kv.split('/')[0]==' к' else int(kv.split('/')[0]))

# площадь 
realt['total_area'] = realt['area'].apply(lambda area: int(''.join(re.findall(r'\w+', area.split('/')[0]))))
realt['living_area'] = realt['area'].apply(lambda area: int(area.split('/')[1]))
realt['kitchen_area'] = realt['area'].apply(lambda area: int(area.split('/')[2])) # 0 - это пропуск

# значения заполнены от балды (999/99/99), замена на средние
realt.loc[991, ['total_area', 'living_area', 'kitchen_area']] = round(realt[realt['rooms'] == 3][['total_area', 'living_area', 'kitchen_area']].mean())
# в примечании общая площадь 83 кв.м.
realt.loc[1651, 'total_area'] = 83

In [6]:
realt['Балкон'].value_counts(dropna=False)

нет                                 893
лоджия застекленная                 301
балкон                              236
лоджия                              202
балкон застекленный                 118
2 лоджии застекленные                60
лоджия застекленная + вагонка        52
балкон застекленный + вагонка        38
2 лоджии                             34
2 балкона                            27
2 лоджии застекленные + вагонка      15
2 балкона застекленные               12
2 балкона застекленные + вагонка     10
2 лоджии 1 застекленная               7
2 балкона 1 застекленный              1
Name: Балкон, dtype: int64

In [7]:
def get_numb_balcony(text):
    if text == 'нет':
        return 0
    elif text.split()[0] in ['лоджия', 'балкон']:
        return 1
    else:
        return 2

In [8]:
realt['Балкон'] = realt['Балкон'].apply(get_numb_balcony)

In [9]:
realt.groupby('Балкон').price_usa.agg(('mean', 'std', 'median', 'min', 'max', 'count'))

,mean,std,median,min,max,count
Балкон,,,,,,
0,620.989922,447.713905,500.0,94,4000,893
1,461.651531,341.980229,370.0,91,3900,947
2,767.897590,476.113834,633.5,98,2500,166


In [10]:
# мебель (частично -> нет)
realt['Мебель'].replace('частично', 'нет', inplace=True)
realt['Мебель'] = realt['Мебель'].apply(lambda furniture: 0 if furniture == 'нет' else 1)

In [11]:
# телефон
realt['Телефон'].replace({'нет': 0,
                          'есть': 1,
                          '2 телефона': 2}, inplace=True)

In [12]:
# Плита: Пропуск -> электрическая
realt['Плита'].replace('Пропуск', 'электрическая', inplace=True)
realt['Плита'] = realt['Плита'].apply(lambda plate: 0 if plate == 'газовая' else 1)

# Ремонт:  хороший ремонт (1) и без ремонта (0). 
# "Пропуск" отнесён пока в "без ремонта" (есть высокие цены, часть может быть новостройкой без ремонта)
realt['Ремонт'] = realt['Ремонт'].apply(lambda repair: 1 if repair in ['евроремонт', 'отличный ремонт'] else 0)

In [13]:
def get_floor_features(text):
    """
    Возвращает (этаж, этажность)
    0 - пропуск (как вариант, пропуск этажности можно заменить на этаж)
    """
    items = re.findall(r'\w+', text)
    if items[-1] in {'этажей', 'этажа'}:
        return (0, int(items[0]))
    elif items[-1] == 'этаж':
        return (int(items[0]), 0)
    else:
        return (int(items[0]), int(items[1]))

In [14]:
# Этаж / этажность
realt['floor'] = realt['Этаж / этажность'].apply(lambda f: get_floor_features(f)[0])
realt['floors'] = realt['Этаж / этажность'].apply(lambda f: get_floor_features(f)[1])

# Замена пропуска в этажности на этаж
realt['floors'].replace(0, realt['floor'], inplace=True)

# Агентство
realt['Агентство'] = realt['Агентство'].apply(lambda agent: 0 if agent=='нет' else 1)

In [16]:
realt.drop(['area',
            'Этаж / этажность',
            'kv'], axis=1, inplace=True)

In [17]:
realt['Планировка'].fillna('Пропуск', inplace=True)
realt['Планировка'].replace({'бреж': 'брежневка',
                             'стал': 'сталинка',
                             'чеш.': 'чешский проект',
                             'таунхаус': 'Пропуск',
                             'малосемейка': 'Пропуск',
                             'таунхаус': 'Пропуск',
                            'пентхаус': 'Пропуск',
                            }, inplace=True)

In [18]:
realt['Тип дома'].fillna('Пропуск', inplace=True)
realt['Тип дома'].replace({'кб': 'каркасно-блочный',
                           'КБ': 'каркасно-блочный',
                           'сб': 'Пропуск'
                            }, inplace=True)

In [19]:
realt['Сан/узел'].unique()

array(['раздельный', nan, 'совмещенный', '2 сан/узла', '3 сан/узла'], dtype=object)

In [20]:
realt['Сан/узел'].fillna(1, inplace=True)
realt['Сан/узел'].replace({'раздельный': 1,
                           'совмещенный': 1.5,
                           '2 сан/узла': 2,
                           '3 сан/узла': 3
                            }, inplace=True)

### Бытовая техника из описания

In [21]:
# Какая техника вообще есть
tech_set = set()
for descr in realt['Бытовая техника']:
    tech_set.update(descr.split(', '))
print(tech_set)

{'', 'душевая кабина', 'кондиционер', 'холодильник', 'свч-печь', 'телевизор', 'компьютерная сеть', 'стиральная машина', 'джакузи', 'посуда', 'музыкальный центр', 'DVD-проигрыватель', 'домашний кинотеатр', 'видеомагнитофон', 'мелкая бытовая техника', 'интернет', 'компьютер', 'выделенная линия в интернет', 'посудомоечная машина', 'высокоскоростной интернет'}


In [22]:
def get_tech_from_descr(text):
    """
    Извлекает бытовую технику из описания
    (мб можно проще)
    """
    result = []
    text = text.lower()
    if 'душевая' in text:
        result.append('душевая кабина')
    if 'интернет' in text:
        result.append('интернет')
    if 'музыкальный центр' in text:
        result.append('музыкальный центр')
    if 'dvd' in text:
        result.append('DVD-проигрыватель')
    if 'кондиционер' in text:
        result.append('кондиционер')
    if 'посудомоечная' in text: 
        result.append('посудомоечная машина')
    if 'джакузи' in text:
        result.append('джакузи')
    if any(tv in text for tv in ['телевизор', 'тв', 'телик']):
        result.append('телевизор')
    if 'свч' in text:
        result.append('свч-печь')
    if 'холодильник' in text:
        result.append('холодильник')
    if 'компьютер' in text:
        result.append('компьютер')
    if 'видеомагнитофон' in text:
        result.append('видеомагнитофон')
    if 'домашний кинотеатр' in text:
        result.append('домашний кинотеатр')
    if 'стиральная' in text:
        result.append('стиральная машина')
    return ', '.join(result)

In [23]:
# Кол-во пропусков для записей с описанием (description)
print("Кол-во пропусков до замены: {}".format(realt[(realt.description != '') & (realt['Бытовая техника'] == '')].shape[0]))
realt.loc[(realt.description != '') & (realt['Бытовая техника'] == ''), 'Бытовая техника'] = realt.loc[(realt.description != '') & (realt['Бытовая техника'] == ''), 'description'].apply(get_tech_from_descr)
print("Кол-во пропусков после замены: {}".format(realt[(realt.description != '') & (realt['Бытовая техника'] == '')].shape[0]))

Кол-во пропусков до замены: 582
Кол-во пропусков после замены: 150


### "Дополнительно" из описания

In [24]:
# Какое "дополнительно" вообще есть
dop_set = set()
for descr in realt['Дополнительно']:
    dop_set.update(descr.split(', '))
print(dop_set)

{'', 'видео-домофон', 'окна во двор', 'встроеные шкафы', 'встроенные шкафы', 'подземный гараж', 'рядом гипермаркет', 'лифт', 'домофон', 'стеклопакеты', 'хороший вид из окна', 'рядом спортивный центр', 'видеодомофон', 'детская площадка', 'рядом рынок', 'металлическая дверь', 'парковка', 'рядом зеленая зона', 'VIP-квартира', 'сигнализация', 'рядом зеленая зона отдыха', 'благоустроенный двор', 'элитная квартира', 'рядом супермаркет', 'рядом магазин', 'гараж', 'охраняемая территория', 'консьержка'}


In [25]:
def get_dop_from_descr(text):
    """
    Извлекает "дополнительно" из описания
    (мб можно проще)
    """
    result = []
    text = text.lower()
    if any(door in text for door in ['металлическая дверь', 'металическая дверь']):
        result.append('металлическая дверь')
    if 'рядом зеленая зона' in text:
        result.append('рядом зеленая зона')
    if any(vid_dom in text for vid_dom in ['видеодомофон', 'видео-домофон']):
        result.append('видеодомофон')
    if any(yard in text for yard in ['благоустроенный двор', 'благоустроеный двор']):
        result.append('благоустроенный двор')
    if 'лифт' in text:
        result.append('лифт')
    if any(garage in text for garage in ['гараж', 'подземный гараж']):
        result.append('гараж')
    if 'парковка' in text:
        result.append('парковка')
    if 'магазин' in text:
        result.append('рядом магазин')
    if 'детская площадка' in text:
        result.append('детская площадка')
    if 'спортивный центр' in text:
        result.append('рядом спортивный центр')
    if any(guard_zone in text for guard_zone in ['охраняемая территория', 'охраняемая територия']):
        result.append('охраняемая территория')
    if 'сигнализация' in text:
        result.append('сигнализация')
    if 'гипермаркет' in text:
        result.append('рядом гипермаркет')
    if 'стеклопакеты' in text:
        result.append('стеклопакеты')
    if any(wardrobe in text for wardrobe in ['встроенные шкафы', 'встроеные шкафы']):
        result.append('встроенные шкафы')
    if 'хороший вид из окна' in text:
        result.append('хороший вид из окна')
    if 'консьержка' in text:
        result.append('консьержка')
    if 'супермаркет' in text:
        result.append('рядом супермаркет')
    if 'домофон' in text:
        result.append('домофон')
    if 'рынок' in text:
        result.append('рядом рынок')
    if 'супермаркет' in text:
        result.append('рядом супермаркет')
    if any(wardrobe in text for wardrobe in ['элитная квартира', 'vip']):
        result.append('VIP-квартира')
    return ', '.join(result)

In [26]:
# Кол-во пропусков для записей с описанием (description)
print("Кол-во пропусков до замены: {}".format(realt[(realt.description != '') & (realt['Дополнительно'] == '')].shape[0]))
realt.loc[(realt.description != '') & (realt['Дополнительно'] == ''), 'Дополнительно'] = realt.loc[(realt.description != '') & (realt['Дополнительно'] == ''), 'description'].apply(get_dop_from_descr)
print("Кол-во пропусков до замены: {}".format(realt[(realt.description != '') & (realt['Дополнительно'] == '')].shape[0]))

Кол-во пропусков до замены: 587
Кол-во пропусков до замены: 274


### Извлечение микрорайонов

In [27]:
import geopandas as gpd
from geopy.distance import vincenty

places = gpd.read_file('../data/geojson/minsk_belarus_places.geojson')

suburbs = places[places['type'] == 'suburb']
for i in realt.index:
    first_point = realt.loc[i, ['latitude', 'longitude']]
    min_dist = suburbs['geometry'].apply(lambda coord: vincenty(first_point, (coord.y, coord.x)).m).agg(('argmin', 'min'))
    realt.at[i, 'Микрорайон'] = suburbs.loc[min_dist['argmin'], 'name']
    #realt.at[i, 'Расст до района'] = min_dist['min']

In [28]:
realt.to_csv('../data/realt_lite_1612.csv', index=False)